In [1]:
%cd ..
!ls

/exp/exp1/acp21rjf/deliberation/speachy
ami		    log.txt		   speachy.egg-info
checkpoints	    model_configs	   sweep_configs
checkpoints_done    model_utils.py	   sweep.yaml
experiment_configs  mwer_rescoring.py	   tedlium
hyps		    non_iid_dataloader.py  tools.py
ipynbs		    pg191kwft.json	   top100_5kb_history_250.pkl
IS_30s_utt.json     __pycache__		   train_LM.py
IS_single_utt	    README.md		   train_lm.sh
lm		    setup.py		   wandb
lm_utils.py	    speachy		   wip


In [2]:
from Levenshtein import distance

In [3]:
import pickle as pkl, numpy as np
from speachy.rescoring.tools import (
        sort_hypothesis_by_recording, 
        order_recordings_by_start_time,
        interpolate
)
from functools import partial, reduce


[NeMo W 2023-01-25 11:41:10 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-01-25 11:41:10 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [4]:
with open('hyps/test_beams_5000.pkl', 'rb') as f:
    test_beams = pkl.load(f)

In [5]:
test_beams = order_recordings_by_start_time(sort_hypothesis_by_recording(test_beams))

In [6]:
def get_edit_distance(hyps, target):
    return list(map(lambda x: distance(x, target) / len(target), hyps))
    
def create_samples_from_recording(recording, num_utterances, num_negatives, max_gap=10.0, shuffle=True):
    samples = []
    prev_end = None
    if shuffle == False:
        np.random.seed(42) # deterministic selection of negatives
    for i, utterance in enumerate(recording):
        start_t, end_t = utterance['meta_data']['timings'].values()
        if prev_end is None or (start_t - prev_end) > max_gap:
            samples.append([]) 
        if len(samples[-1]) >= num_utterances:
            samples.append([])
        hyps = utterance['beams'][0]
        hyps = list(map(lambda x: x['text'], list(hyps.values())))
        target = utterance['targets'][0]
        hyps = list(filter(lambda el:el != target, hyps))
        hyps = np.random.choice(hyps, min(num_negatives, len(hyps)), replace=False).tolist()
        examples = [target] + hyps
        error_rates = get_edit_distance(examples, target)
        samples[-1].append((examples, error_rates))
        prev_end = end_t
        
    return samples

In [7]:
def create_dataset_samples(recordings, num_utterances, num_negatives, max_gap=10.0, shuffle=True):
    samples = []
    for recording in recordings.keys():
        samples += create_samples_from_recording(recordings[recording], num_utterances, num_negatives, max_gap, shuffle)
    if shuffle:
        np.random.shuffle(samples) # shuffle samples
    return samples

In [8]:
def tokenize_and_pad(utterances, tokenizer):
    tokenized = [tokenizer.text_to_ids(utt) for utt in utterances]
    max_len = max(map(len, tokenized))
    return np.array([utt + [0] * (max_len - len(utt)) for utt in tokenized])

In [9]:
flatten_nested_list = lambda l: [item for sublist in l for item in sublist]

In [10]:
def get_sub_batches(batch, tokenizer):
    proc_utts = lambda utts: tokenize_and_pad(flatten_nested_list(utts), tokenizer)
    sub_batches = []
    max_len = max(map(len, batch))
    for i in range(max_len):
        sub_batches.append({
            'utterances': [],
            'scores': [],
            'lengths': [],
        })
        for el in batch:
            if len(el) > i:
                sub_batches[-1]['utterances'].append(el[i][0])
                sub_batches[-1]['scores'].append(el[i][1])
                sub_batches[-1]['lengths'].append(len(el[i][0]))
            else:
                sub_batches[-1]['utterances'].append(-1)
                sub_batches[-1]['scores'].append(-1)
                sub_batches[-1]['lengths'].append(-1)
    non_empty_indices = np.arange(len(sub_batches[0]['lengths']))

    for i, sub_batch in enumerate(sub_batches):
        sb_utts, sb_scores, sb_lengths = np.array(sub_batch['utterances'], dtype=object), \
            np.array(sub_batch['scores'], dtype=object), np.array(sub_batch['lengths'], dtype=object)
        non_empty = non_empty_indices[sb_lengths != -1]
        # slice based on non empty of previous sub batch
        prev_fetch = None
        if i != 0:
            prev_lengths = sub_batches[i-1]['lengths']
            prev_non_empty = sub_batches[i-1]['non_empty']
            diff_from = ((prev_lengths != -1) == (sb_lengths != -1))[prev_non_empty]
            prev_fetch = np.arange(len(prev_non_empty))[diff_from]
        sub_batches[i] = {
            'utterances': sb_utts,
            'scores': sb_scores,
            'lengths': sb_lengths,
            'non_empty': non_empty,
            'prev_fetch': prev_fetch 
        }
    
    for i, sub_batch in enumerate(sub_batches):
        sub_batches[i] = {
            'utterances': proc_utts(sub_batch['utterances'][sub_batch['non_empty']].tolist()),
            'scores': sub_batch['scores'][sub_batch['non_empty']],
            'lengths': sub_batch['lengths'][sub_batch['non_empty']],
            'prev_fetch': sub_batch['prev_fetch'] # indices to fetch the states from previous sub batch
        }
    return sub_batches

In [11]:
def sampler(samples, batch_size, tokenizer, shuffle=True):
    sample_indices = np.arange(len(samples))
    np.random.shuffle(sample_indices) if shuffle else None
    batches = []
    for i in range(0, len(samples), batch_size):
        yield get_sub_batches([samples[i] for i in sample_indices[i:i+batch_size]], tokenizer)
        

In [ ]:
utterance_list = [[utt[0] for utt in utt_in_batch] for utt_in_batch in s1]
score_list = [[utt[1] for utt in utt_in_batch] for utt_in_batch in s1]

In [532]:
unpack_nested_list = lambda lst: reduce(lambda x,y: x+y, lst)

In [533]:
len(samples)

15

In [534]:
sbs[-40]['non_empty'] == sbs[-39]['non_empty']

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_217678/1012350713.py:1 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_217678/1012350713.py'                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'non_empty'

In [535]:
np.arange(9) == np.arange(3)

[NeMo W 2023-01-24 17:34:20 nemo_logging:349] /tmp/ipykernel_217678/3944724721.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
      np.arange(9) == np.arange(3)
    


False

In [385]:
ix = -2
print(sbs[ix]['lengths'])
print(len(sbs[ix]['utterances']))
print(sbs[ix]['prev_fetch'])

[6]
1
[0]


In [425]:
len(s1[0]['utterances'])

30

In [14]:
import tools

In [15]:
tk = tools.load_tokenizer('./tedlium/tokenizers/tokenizer_spe_bpe_v1000/tokenizer.model')

In [16]:
def tokenize_and_pad(utterances, tokenizer):
    tokenized = [tokenizer.text_to_ids(utt) for utt in utterances]
    max_len = max(map(len, tokenized))
    return np.array([utt + [0] * (max_len - len(utt)) for utt in tokenized])

In [17]:
tcol(cuts=None,text=(s1[0]['utterances'][-5], s1[0]['utterances'][-5]))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_552617/4219463649.py:1 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_552617/4219463649.py'                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'tcol' is not defined

In [527]:
np.array([['f','z'],['helo']]).tolist()

[NeMo W 2023-01-24 17:33:01 nemo_logging:349] /tmp/ipykernel_217678/3555533298.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
      np.array([['f','z'],['helo']]).tolist()
    


[['f', 'z'], ['helo']]

In [ ]:
s1[2]['utterances']

In [40]:
s1[0]['utterances'][0]

array([ 41, 440, 103, 158,   4, 279, 448,   6, 350,  53, 170,   4, 279,
        16,  53,  27, 438, 106, 217, 987,  20,   4, 279, 351,  37,  42,
       146, 104, 299,   5, 208, 979, 125,   4, 304,  34,  60, 293, 973,
       482,  20, 102,   6, 759,  33,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0])

In [19]:
s1 = next(sampler(samples, 5, tk))

In [18]:
samples = create_dataset_samples(test_beams, num_negatives=5, num_utterances=500)

In [105]:
hyps

array(["i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my the sars handy never a' writing anything but",
       "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my theisarres handy never im writing anything but",
       "i'd like to share with you a discovery that i made a few months ago while writing an article for italian wired i always keep my the sars handy never are and writing anything but",
       "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my f the sars handy never ionm writing anything but",
       "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my the sarres handy i never a writing anything but"],
      dtype='<U188')

In [106]:
hyps = list(map(lambda el:el['text'],list(test_beams['AimeeMullins_2009P'][0]['beams'][0].values())))
target = test_beams['AimeeMullins_2009P'][0]['targets'][0]
hyps = list(filter(lambda el:el != target, hyps))
# select 5 random hyps
hyps = np.random.choice(hyps, min(5, len(hyps)), replace=False).tolist()
hyps = hyps + [target]

In [97]:
import torch

In [123]:
list(zip(get_edit_distance(hyps, target), hyps))

[(0.096045197740113,
  "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my f sars handy i've never writing anything but"),
 (0.0903954802259887,
  "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my the f sars handy i never writing anything but"),
 (0.1016949152542373,
  "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my f the sars handy i never i a and writing anything but"),
 (0.096045197740113,
  "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my th the sars handy i never writing anything but"),
 (0.06779661016949153,
  "i'd like to share with you a discovery that i made a few months ago while writing an article for the italian wired i always keep my the sars han

In [117]:
(torch.tensor(get_edit_distance(hyps, target))).mean(0)

tensor(0.0753)

In [115]:
(torch.tensor(get_edit_distance(hyps, target))) - (torch.tensor(get_edit_distance(hyps, target))).mean(0)

tensor([ 0.0207,  0.0151,  0.0264,  0.0207, -0.0075, -0.0753])